### Tugas

Pada folder data, terdapat dataset jamur yang kita gunakan pada materi Decision Tree. Berdasarkan dataset yang sama, bandingkan peforma antara algoritma DT dan RandomForest. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik. 

### Import Library

In [99]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # import DT
from sklearn.ensemble import RandomForestClassifier # import RandomForest
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

### Persiapan Data

In [100]:
# Load data
df = pd.read_csv('mushrooms.csv')

df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [101]:
# Cek kolom null
df.isnull().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

In [102]:
# Seleksi fitur

# Slice dataframe mulai dari kolom 'radius_mean' sampai 'fractal_dimension_worst'
X = df.iloc[:,2:-1]
y = df['class']
y = y.map({'M':1, 'B':0}) # Encode label

# Cek jumlah fitur dan instance
X.shape

(8124, 20)

### Split data training dan testing

In [103]:
# Encoding
# Fungsi encoding yang akan digunakan adalah LabelEncoder
# Hal ini karena kita hanya mengganti nilai variabel dari nama berupa string menjadi angka. Sama halnya dengan label

from sklearn.preprocessing import LabelEncoder

# Inisiasi label encoder
encode = LabelEncoder()

# Terpakan label encoder
df['class'] = encode.fit_transform(df['class'])
df['cap-shape'] = encode.fit_transform(df['cap-shape'])
df['cap-surface'] = encode.fit_transform(df['cap-surface'])
df['cap-color'] = encode.fit_transform(df['cap-color'])
df['bruises'] = encode.fit_transform(df['bruises'])

df['odor'] = encode.fit_transform(df['odor'])
df['gill-attachment'] = encode.fit_transform(df['gill-attachment'])
df['gill-spacing'] = encode.fit_transform(df['gill-spacing'])
df['gill-size'] = encode.fit_transform(df['gill-size'])
df['gill-color'] = encode.fit_transform(df['gill-color'])

df['stalk-shape'] = encode.fit_transform(df['stalk-shape'])
df['stalk-root'] = encode.fit_transform(df['stalk-root'])
df['stalk-surface-above-ring'] = encode.fit_transform(df['stalk-surface-above-ring'])
df['stalk-surface-below-ring'] = encode.fit_transform(df['stalk-surface-below-ring'])
df['stalk-color-above-ring'] = encode.fit_transform(df['stalk-color-above-ring'])

df['stalk-color-below-ring'] = encode.fit_transform(df['stalk-color-below-ring'])
df['veil-type'] = encode.fit_transform(df['veil-type'])
df['veil-color'] = encode.fit_transform(df['veil-color'])
df['ring-number'] = encode.fit_transform(df['ring-number'])
df['ring-type'] = encode.fit_transform(df['ring-type'])

df['spore-print-color'] = encode.fit_transform(df['spore-print-color'])
df['population'] = encode.fit_transform(df['population'])
df['habitat'] = encode.fit_transform(df['habitat'])
# Cek hasil
df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,3,2,4,0,5,0,0,0,11,...,2,5,5,0,1,1,4,0,1,2
8120,0,5,2,4,0,5,0,0,0,11,...,2,5,5,0,0,1,4,0,4,2
8121,0,2,2,4,0,5,0,0,0,5,...,2,5,5,0,1,1,4,0,1,2
8122,1,3,3,4,0,8,1,0,1,0,...,1,7,7,0,2,1,0,7,4,2


In [104]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

### Training Decision Tree

In [105]:
# Secara default, DecisionTreeClassifier dari scikit-learn akan menggunakan nilai "Gini" untuk kriteria
# Terdapat beberapa "hyperparamater" yang dapat digunakan. Silahka baca dokumentasi
# Pada kasus ini kita akan menggunakan parameter default
dt = DecisionTreeClassifier(max_depth=8, random_state=1)

# Sesuaikan dt ke set training
dt.fit(X_train, y_train)

# Memprediksi label set test
y_pred_dt = dt.predict(X_test)

#  menghitung set accuracy

acc_dt = accuracy_score(y_test, y_pred_dt)
# calculate accuracy_score for training data:
print(accuracy_score(y_train, dtc.predict(X_train)))
# calculate accuracy_score for test data:
print(accuracy_score(y_test, dtc.predict(X_test)))



ValueError: could not convert string to float: 'f'

### Training RandomForest

In [ ]:
# Pada kasus kali ini kita akan menggunakan seluruh parameter default dari RandomForest
# Untuk detail parameter (hyperparameter) silahkan cek dokumentasi

rf = RandomForestClassifier(n_estimators=10, random_state=1)

# Sesuaikan dt ke set training
rf.fit(X_train, y_train)

# Memprediksi label set test
y_pred_rf = rf.predict(X_test)

#  menghitung set accuracy
acc_rf = accuracy_score(y_test, y_pred_rf)
print("Test set accuracy: {:.2f}".format(acc_rf))
print(f"Test set accuracy: {acc_rf}")

Test set accuracy: 0.96
Test set accuracy: 0.956140350877193


### Tugas

Pada folder data, terdapat dataset jamur yang kita gunakan pada materi Decision Tree. Berdasarkan dataset yang sama, bandingkan peforma antara algoritma DT dan RandomForest. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik. 